# DB Cleaning Book To Use on pre 2021 Box Score File
- these files don't have advanced metrics table 

- code adapted from original clean_and_transform workbook

In [35]:
## Dependencies

import pandas as pd
import numpy as np
import sqlite3
import os


# Path to master roster file
roster_path = os.path.join(os.getcwd(), "..", "data", 'rosters', 'all_time_combined_roster.csv')
# Read in the master roster file
roster = pd.read_csv(roster_path)
# roster.head()

# Set Path to db file 
db_path = os.path.join(os.getcwd(), "..", 'TEMP', 'Box_Scores_2016_v2.db')

# Connect to the database
conn = sqlite3.connect(db_path)
# Create a cursor
cur = conn.cursor()
# Print table names to check connection
# cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
# print(cur.fetchall())



C:\Users\jbanc\AppData\Local\Temp\ipykernel_20904\927307407.py:12: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  roster = pd.read_csv(roster_path)


In [36]:
####################
## Set the season 

season_year = 2016

####################

# Filter the roster to the season of interest
roster_df = roster[roster['Season'] == season_year]


## Create dictionary of Team Primary Names to Abbreviations

- Needed to Add IVY teams becuase of low amount of games. will have to do for harvard, yale, ect next week




In [37]:
## Create dataframe from SQL query
df = pd.read_sql_query("SELECT * FROM scoring_summary", conn)

# Function to count the occurrences of primary team names for unmatched abbreviations
def count_primary_names_for_abbreviation(abbreviation):
    filtered_rows = df[df['Team'] == abbreviation]
    team_counts = {}
    
    for _, row in filtered_rows.iterrows():
        teams = row['game_id'].split('-')[-2:]
        for team in teams:
            if team not in team_counts:
                team_counts[team] = 0
            team_counts[team] += 1
            
    return team_counts


# Attempt to match abbreviations to primary names based on substrings and common naming conventions
matched_dict = {}
unmatched_abbreviations = []

for abbreviation in df['Team'].unique():



# Match the abbreviation to the primary team name with the highest occurrence

    team_counts = count_primary_names_for_abbreviation(abbreviation)
    # Get the team with the highest count
    matched_team = max(team_counts, key=team_counts.get)
    matched_dict[abbreviation] = matched_team

# matched_dict

# Manually fix the unmatched abbreviations - IVY League Teams with no of very few games throw a wrench in the above method
# Brown: Brown
# Cornell: Cornell

# Make those substitutions
matched_dict['Brown'] = 'Brown'
matched_dict['Cornell'] = 'Cornell'
# yale
matched_dict['Yale'] = 'Yale'
# princeton
matched_dict['Princeton'] = 'Princeton'

# harvard
matched_dict['Harvard'] = 'Harvard'
# columbia
matched_dict['Columbia'] = 'Columbia'

# dartmouth
matched_dict['Dartmouth'] = 'Dartmouth'

# penn
matched_dict['Penn'] = 'Penn'

# BC
matched_dict['BC'] = 'Boston College'



# matched_dict

In [38]:
# Print database table names and check to see if they are locked
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())



[('goalie_stats',), ('game_details',), ('player_stats',), ('linescore',), ('penalty_summary',), ('scoring_summary',)]


## Add Home and Away Columns to game_details table

In [39]:
# Step 1: Read the game_details table into a DataFrame
df_game_details = pd.read_sql("SELECT * FROM game_details", conn)

# Step 2: Create new columns for Home and Away Teams by parsing Game_ID
df_game_details['Away_Team'] = df_game_details['game_id'].apply(lambda x: x.split('-')[3])
df_game_details['Home_Team'] = df_game_details['game_id'].apply(lambda x: x.split('-')[4])

# Step 3: Write this updated DataFrame back to the game_details table
df_game_details.to_sql('game_details', conn, if_exists='replace', index=False)


1125

## Clean up The Column Names and extra header rows in the Player Stats table

In [40]:
############ 'Pt.' should be 'Pts' and '+/-' should be 'plus_minus'
#################################
player_stats_df = pd.read_sql_query("SELECT * FROM player_stats", conn)

# Define a dictionary for column renaming
column_renames = {
    'Pt.': 'Pts',
    '+/-': 'plus_minus'
}

# Rename columns based on the dictionary
player_stats_df.rename(columns=column_renames, inplace=True)


# Drop rows where Team name is in the Player column
player_stats_df = player_stats_df[player_stats_df['Team'] != player_stats_df['Player']]

## Change the Column names to be easy to work with
############ 'Pt.' should be 'Pts' and '+/-' should be 'plus_minus'
#################################
player_stats_df = pd.read_sql_query("SELECT * FROM player_stats", conn)

if 'Pt.' in player_stats_df.columns:
    player_stats_df.rename(columns={'Pt.': 'Pts'}, inplace=True)
else:
    print("Column 'Pt.' not found.")

if '+/-' in player_stats_df.columns:
    player_stats_df.rename(columns={'+/-': 'plus_minus'}, inplace=True)
else:
    print("Column '+/-' not found.")

print(len(player_stats_df))

# Drop rows if Team name is in the player column
# If ['Team'] is the same as ['Player'] then drop that row
player_stats_df = player_stats_df[player_stats_df['Team'] != player_stats_df['Player']]

# add the dataframe back to the database
player_stats_df.to_sql('player_stats', conn, if_exists='replace', index=False)

# print(len(player_stats_df))
#################################
# player_stats_df.head()


Column 'Pt.' not found.
Column '+/-' not found.
42930


42930

In [41]:
## Add The primary team names to the linescores table
# Read the linescores table into a DataFrame
df_linescores = pd.read_sql("SELECT * FROM linescore", conn)

# Apply the dictionary to the Team column
df_linescores['Team'] = df_linescores['Team'].apply(lambda x: matched_dict[x])

df_linescores.head()

,Team,goals1,goals2,goals3,goals4,goalsT,shots1,shots2,shots3,shots4,...,PPG,PPO,FOW,FOL,FOW%,goals5,goals6,shots5,shots6,game_id
0,Army,0,1,1,0,2,13,5,7,2,...,2,13,28,36,43.750000,0,0,0,0,2016-10-01-Army-Colgate
1,Colgate,1,1,0,0,2,17,5,5,3,...,1,9,36,28,56.250000,0,0,0,0,2016-10-01-Army-Colgate
2,Michigan Tech,0,0,0,0,0,7,10,4,0,...,0,4,26,30,46.428571,0,0,0,0,2016-10-01-Michigan Tech-Minnesota Duluth
3,Minnesota Duluth,2,3,1,0,6,9,11,7,0,...,3,6,30,26,53.571429,0,0,0,0,2016-10-01-Michigan Tech-Minnesota Duluth
4,Alabama Huntsville,1,2,1,0,4,9,10,8,0,...,1,2,31,37,45.588235,0,0,0,0,2016-10-02-Alabama Huntsville-Ferris State


# Penalty Table & Scoring Summary

In [42]:
## Add The primary team names to the linescores table
# Read the linescores table into a DataFrame

df_penalty = pd.read_sql("SELECT * FROM penalty_summary", conn)

# Apply the dictionary to the Team column
#$ Skip if not found

df_penalty['Team'] = df_penalty['Team'].apply(lambda x: matched_dict[x])

    

# Apply same method to scorring_summary:
df_scoring = pd.read_sql("SELECT * FROM scoring_summary", conn)
df_scoring['Team'] = df_scoring['Team'].apply(lambda x: matched_dict[x])

# Add Home and Away Team columns to scoring_summary
df_scoring['Away_Team'] = df_scoring['game_id'].apply(lambda x: x.split('-')[3])
df_scoring['Home_Team'] = df_scoring['game_id'].apply(lambda x: x.split('-')[4])


## Add each table back to database
# Write the updated linescores DataFrame back to the linescore table
df_linescores.to_sql('linescore', conn, if_exists='replace', index=False)

# Write the updated penalty DataFrame back to the penalty_summary table
df_penalty.to_sql('penalty_summary', conn, if_exists='replace', index=False)

# Write the updated scoring DataFrame back to the scoring_summary table
df_scoring.to_sql('scoring_summary', conn, if_exists='replace', index=False)

6492

## CREATE A NEW TABLE WITH AGGRIGATED PLAYER STATS YEAR TO DATE

In [43]:
# Use player_stats_df from here on, instead of running another SQL query.
df_player_stats = player_stats_df.copy()


# Clean up the name format in player_stats for easier matching
# Replace the non-breaking space with a regular space
df_player_stats['Clean_Player'] = df_player_stats['Player'].apply(lambda x: x.replace('\xa0', ' '))

# Remove rows where Player is the team name (e.g., "Michigan State")
df_player_stats_cleaned = df_player_stats[df_player_stats['Player'] != df_player_stats['Team']]

# Convert relevant columns to integers for correct aggregation
cols_to_convert = ['G', 'A', 'Pts', 'plus_minus', 'Sh', 'PIM']
for col in cols_to_convert:
    df_player_stats_cleaned[col] = pd.to_numeric(df_player_stats_cleaned[col], errors='coerce')

# Aggregate the data for year-to-date stats
# Add a column for counting the number of games each player has played
agg_player_stats_corrected_with_games = df_player_stats_cleaned.groupby(['Clean_Player', 'Team']).agg({
    'G': 'sum',
    'A': 'sum',
    'Pts': 'sum',
    'plus_minus': 'sum',
    'Sh': 'sum',
    'PIM': 'sum',
    'game_id': 'count'  # Counting the number of unique Game_IDs for each player
}).reset_index()

# Rename the Game_ID column to Games_Played
agg_player_stats_corrected_with_games.rename(columns={'game_id': 'Games_Played'}, inplace=True)

# Save the updated aggregated data back to the database, replacing the existing table
agg_player_stats_corrected_with_games.to_sql('player_stats_ytd', conn, if_exists='replace', index=False)

# Verify by loading some sample data from the updated table
sample_updated_ytd = pd.read_sql_query("SELECT * FROM player_stats_ytd LIMIT 5;", conn)
sample_updated_ytd


,Clean_Player,Team,G,A,Pts,plus_minus,Sh,PIM,Games_Played
0,A.J. Fossen,Clarkson,5,4,9,-1,22,44,30
1,A.J. Reid,Air Force,9,8,17,0,91,27,42
2,Aaron Hadley,Western Michigan,2,9,11,3,44,12,40
3,Aaron Herdt,Alaska,0,1,1,-1,11,2,12
4,Aaron Nelson,Minnesota State,0,0,0,0,0,0,9


## Add the Roster data to the Database

In [44]:
################## SET THE ROSTER DATAFRAME TO THE CORRECT YEAR ####################
## MATCH THE DATAFRAME NAMES
df_master_roster = roster_df.copy()


# Clean up the name formats for joining
# Master roster: Convert "Last Name, First Name" to "First Name Last Name"
# df_master_roster['Clean_Name'] = df_master_roster['Player'].apply(lambda x: ' '.join(reversed(x.split(', '))))

# Rename Player to Clean_Name
df_master_roster.rename(columns={'Player': 'Clean_Name'}, inplace=True)
# Rename School to Team
df_master_roster.rename(columns={'School': 'Team'}, inplace=True)

# Clean up the Team column, remove '-' and replace with ' '
# df_master_roster['School'] = df_master_roster['Team'].apply(lambda x: x.replace('-', ' '))

## If there are an period in the column names, remove them
df_master_roster.columns = df_master_roster.columns.str.replace('.', '')

### Finally add the roster to the database as it's own table

df_master_roster['SeasonYear'] = season_year

# Save the roster data as a new table in the database
roster_table_name = 'master_roster'
df_master_roster.to_sql(roster_table_name, conn, if_exists='replace', index=False)
############################################################

# Verify by listing all the tables in the database again
tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = conn.execute(tables_query).fetchall()
table_names_updated = [table[0] for table in tables]
table_names_updated





['goalie_stats',
 'game_details',
 'player_stats',
 'linescore',
 'penalty_summary',
 'scoring_summary',
 'player_stats_ytd',
 'master_roster']